In [ ]:
from IPython.display import Image
Image('../input/predicting-wins-in-lol/1200px-League_of_Legends_2019_vector.svg.png')

# Introduction

League of Legends (LoL) is a competitive, online-multiplayer video game created by Riot Games in 2009. Two teams composed of five champions (these are the avatars of the game) each are placed on Summoner's Rift and are tasked with destroying the enemy team's nexus. Along the way, each summoner (player in control of the champion) can collect gold and gain experience by destroying towers or killing enemy champions or minions. The first team to destroy a nexus wins.

# Data

This project uses 10-minutes of data collected from over nine-thousand unique games. The games are all ranked games at high elo (Diamond 1 to Masters).

This dataset contains 40 columns including the "gameId" column which is a unique identifier. The target column is called "blueWins". The blue team wins the gain if "blueWins" is 1, otherwise the red team wins.

The remaining 38 columns are split into 19 columns describing the characteristics of the game for each team. Among these 38 columns are: "blueWardsDestroyed", "redWardsPlaced", "blueAvgLevel", and "redTowersDestroyed".

# Purpose

The purpose of this project is to build an accurate machine learning model that can determine whether a team will win a game based on the first 10 minutes of the game. From this insight, we can then look at the characteristics of the winning team.

It is important to note that this model cannot be 100% accurate because there is a lot that happens after the first 10 minutes of the game that can quickly change the tides of the game.

# Analysis

Now we take a look at our dataset, we begin by importing all the neccesary modules.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import plot_confusion_matrix, confusion_matrix

from xgboost import XGBClassifier

from scipy.stats import skew
from scipy.stats import randint, truncnorm, uniform

import warnings
warnings.filterwarnings('ignore')

Next we import the data and have a quick view at the first 5 elements.

In [ ]:
data = pd.read_csv('../input/league-of-legends-diamond-ranked-games-10-min/high_diamond_ranked_10min.csv')
data.head()

We can also see a summary of the data as follows.

In [ ]:
data.info()

It seems like there are no missing values, but we run the following line to be sure.

In [ ]:
data.isna().values.any()

We have no missing values! Next we get a quick visualization of the data using histograms.

In [ ]:
data.hist(bins = 50, figsize = (20,40))
plt.show()

It appears that most of the data is normally distributed!

## Baseline Model

Now I want to create a baseline model and the goal will be to pass the baseline's accuracy. The baseline model will use LogisticRegression.

First, we need to split the data into test/train sets and remove the target columns. We will train 80% of the data.

In [ ]:
train_set, test_set = train_test_split(data, test_size = 0.2, random_state = 2)

# I will create a small function that separates the target variable

def get_target(data_set):
    return data_set.drop('blueWins', axis = 1), data_set['blueWins']

X_train, y_train = get_target(train_set)
X_test, y_test = get_target(test_set)

In [ ]:
L_reg = LogisticRegression()
X_train_copy = X_train.copy() # I want to avoid changing the original
scores = cross_val_score(L_reg, X_train_copy, y_train, scoring = 'accuracy', cv = 10)

print(scores.mean())

Looks like the baseline model has an accuracy of 50%. This is not the best model, but that is okay. Now the goal is to pass 50%.

# Feature Engineering

Now I would like to try some feature engineering. This will include adding new features, fixing skew, scaling the data and removing highly-correlated features.

### Creating new Features

We begin by adding new features. In the dataset, there are four attributes that relate to vision "(blue/red)WardsPlaced" and "(blue/red)WardsDestroyed". I will create a new feature "(blue/red)VisionScore", which demonstrates vision dominance. A team has better vision when they place their own wards and destroy enemy wards.

In [ ]:
def add_attributes(X):
    X['blueVisionScore'] = X['blueWardsPlaced'] + X['redWardsDestroyed']
    X['redVisionScore'] = X['redWardsPlaced'] + X['blueWardsDestroyed']
    return X

X_train_copy = X_train.copy()
X_train_tr = add_attributes(X_train_copy)
print(X_train_tr.columns)

The goal is to place this function into a pipeline, so I will create a class that has fit and transform methods.

In [ ]:
class attributes_adder(BaseEstimator, TransformerMixin):
    def fit(self, X):
        return self
    def transform(self, X):
        return add_attributes(X)

### Fixing High Skew

Now we will create a function that finds the skew. By looking at the histograms above, we can see that some of the data is not normal. So I will exclude that when calculating the skew. We will attempt to fix the skew if the absolute value of the skew is greater than 0.5.

In [ ]:
def intersection(x,y):
    return [val for val in x if val in y]

def find_high_skew(X):
    non_norm = ['gameId','blueFirstBlood','blueEliteMonsters','blueHeralds','blueDragons','blueTowersDestroyed',
              'redFirstBlood','redEliteMonsters','redHeralds','redDragons','redTowersDestroyed']
    non_norm = intersection(list(X.columns), non_norm)
    temp = X.drop(non_norm, axis = 1)
    cols = []
    skews = []
    for col in temp.columns:
        if abs(skew(temp[col])) > 0.5: # we will transform the attributes with high skew
            cols.append(col)
            skews.append(skew(temp[col]))
    return cols, skews

In [ ]:
X_train_copy = X_train.copy()

cols, skews = find_high_skew(X_train_copy)

print("Columns with skew > 0.5")
    
X_train_copy[cols].hist(bins = 50, figsize = (20,40))
plt.show()

Just as before, I will create a custom class for this skew function that will later be put into a pipeline.

To fix the skew, we will attempt a square-root or log transformation.

In [ ]:
class fix_skew(BaseEstimator, TransformerMixin):
    def __init__(self, try_sqrt = True):
        self.try_sqrt = try_sqrt
    def fit(self,X):
        return self
    def transform(self,X):
        skewed_cols, _ = find_high_skew(X)
        if self.try_sqrt:
            for col in skewed_cols:
                X[col] = np.sqrt(X[col]) # try the square root tranformation
        else:
            for col in skewed_cols:
                X[col] = np.log(1 + X[col]) # otherwise, try a log tranformation
        return X

# now we test
testClass = fix_skew() # log transformation
X_train_copy = X_train.copy()
X_train_log = testClass.transform(X_train_copy)

print("Histograms after fixing the skew using a Log transformation.")
X_train_log[cols].hist(bins = 50, figsize = (20,40))
plt.show()

In [ ]:
testClass = fix_skew(try_sqrt = True) #square-root
X_train_copy = X_train.copy()
X_train_sqrt = testClass.transform(X_train_copy)

print("Histograms after fixing the skew using a Square Root transformation.")
X_train_sqrt[cols].hist(bins = 50, figsize = (20,40))
plt.show()

In [ ]:
cols1, skews1 = find_high_skew(X_train_log)
cols2, skews2 = find_high_skew(X_train_sqrt)

print("Leftover high skew after log transformation")
print(cols1, skews1)

print("==========================\n")
print("Leftover high skew after square root transformation")
print(cols2, skews2)

It appears that both tranformations worked equally well!

### Removing Highly Correlated Features

Next, I would like to remove columns that are highly correlated. 

In [ ]:
def remove_corr(X, corrTol = 0.75):
    colToDrop = set()
    for col in X.columns:
        corr = X.corr()
        corrDict = corr[col].to_dict()
        del(corrDict[col])
    for key in corrDict:
        if abs(corrDict[key]) > corrTol:
            colToDrop.add(key)
    colToDrop = list(colToDrop)
    X.drop(colToDrop, axis = 1, inplace = True)
    X.drop('gameId', axis = 1, inplace = True) # gameId does not contribute, so we remove it here.
    return X

X_train_copy = X_train.copy()

X_train_tr = remove_corr(X_train_copy)
print(X_train_tr.columns)

Now we create a custom class.

In [ ]:
class removeCorr(BaseEstimator, TransformerMixin):
    def __init__(self,corrTol = 0.75):
        self.corrTol = corrTol
    def fit(self, X):
        return self
    def transform(self, X):
        return remove_corr(X, self.corrTol)

## Create a Pipeline

Now, we will add all the classes into a pipeline. The final step in the pipeline will be to scale the data using sklearn's StandardScaler. First, we want the StandardScaler to return a dataframe. So I will create a class that will do that.

In [ ]:
class scaleData(BaseEstimator, TransformerMixin):
    def fit(self, X):
        return self
    def transform(self,X):
        tmp = X.copy()
        X = StandardScaler().fit_transform(X)
        return pd.DataFrame(X, index = tmp.index, columns = tmp.columns)

myPipe = Pipeline([
    ('attributes_adder', attributes_adder()),
    ('fix_skew', fix_skew()),
    ('removeCorr',removeCorr(corrTol = 0.95)),
    ('scaler', scaleData())
])

X_train_copy = X_train.copy()
X_train_tr = myPipe.fit_transform(X_train_copy)

X_train_tr.hist(bins = 50, figsize = (20,40))
plt.show()

# Model Building

Now it is time to train models! I will first use LogisticRegression, DecisionTreeClassifier, RandomForestClassifier, and GradientBoostingClassifier with default parameters. Then, I will chose the most accurate model and use RandomizedSearchCV to try to improve the accuracy further.

In [ ]:
def trainModel(models, X, y):
    modelScores = {}
    for model in models:
        scores = cross_val_score(model, X, y, scoring = 'accuracy', cv = 10)
        avgScore = scores.mean()
        modelScores[model] = avgScore
    return modelScores

models = [LogisticRegression(random_state = 1), 
          DecisionTreeClassifier(random_state = 2),
          RandomForestClassifier(random_state = 3),
          GradientBoostingClassifier(random_state = 4)
         ]

modelScores = trainModel(models, X_train_tr, y_train)

for model in modelScores:
    print(model, modelScores[model])

### Optimizing Best Model

LogisticRegression is the best model without any hyperparameter tuning. Now we will use RandomizedGridSearch to find the optimal parameters.

In [ ]:
param_distributions = [{
    'C': uniform(0.01,2),
    'max_iter': [val*10 for val in range(5,16)],
    'n_jobs' : [val*10 for val in range(9,20)],
    'l1_ratio': truncnorm(a = 0, b = 1, loc = 0.5, scale = 0.1),
    'intercept_scaling': truncnorm(a = 0, b = 2, loc = 1, scale = 0.5)
}]

LR = LogisticRegression(random_state = 1)
grid_LR = RandomizedSearchCV(LR, param_distributions, cv = 10, scoring = 'accuracy', n_iter = 20)
model = grid_LR.fit(X_train_tr, y_train)
print(model.best_score_)
print(model.best_params_)
preds = model.predict(X_train_tr)


Using RandomizedSearchCV, we increased our accuracy by a small amount.
Next, we predict using our fitted model and assess the accuracy using a confusion matrix.

In [ ]:
X_test_copy = X_test.copy()
X = myPipe.fit_transform(X_test_copy)
plot_confusion_matrix(model, X, y_test)
plt.show()

Below, we see the accuracy of the model.

In [ ]:
X_test_copy = X_test.copy()
X = myPipe.fit_transform(X_test_copy)
preds = model.predict(X)
tn, fp, fn, tp = confusion_matrix(y_true = y_test, y_pred = preds).ravel()

acc = (tn + tp)/(tn + fp + fn + tp)

print('Accuracy: {:.2f}%'.format(acc*100))

# Analyzing Predictions

Next, I would like to see the characteristics of the predicted wins.

In [ ]:
X = X_test.copy()
index = X.index
y = pd.Series(data = preds, index = index, name = 'blueWins')
data = pd.concat([X,y], axis = 1)
blueWins = data[data['blueWins'] == 1]
blueLoses = data[data['blueWins'] == 0]

# I only want to keep the blue attributes, so I'll drop red columns, gameId, and the 'blueWins' column
delCols = []
for col in blueWins.columns:
    if 'red' in col:
        delCols.append(col)
delCols += ['blueWins','gameId']

blueWins.drop(delCols, axis = 1, inplace = True)
blueLoses.drop(delCols, axis = 1, inplace = True)

In [ ]:
for col in blueWins.columns:
    fig, ax = plt.subplots(figsize = (10,10))
    plt.hist(blueWins[col], bins = 50, label = 'Wins', alpha = 0.5)
    plt.hist(blueLoses[col], bins = 50, label = 'Loses', alpha = 0.5)
    plt.title(col)
    plt.legend(loc = 'upper right')
    plt.show()

From the histograms above, we see a few key characteristics between wins and loses:

* First blood leads to a win half of the time.
* The team with the most kills is more likely to win.
* The team with the lease deaths is more likely to win.
* The team with the most assists is more likely to win. (League of Legends is a team sport!)
* The team the most elite monster kills is more likely to win.
* The team with the most dragon kills is more likely to win.
* The team that gets the rift herald is more likely to win.
* The team with more gold is most likely to win.
* The team with a higher average level is more likely to win.
* The team with the higher experience is more likely to win.
* The team with the most minions kills is more likely to win.

Most of these characteristics go hand in hand. For example, if your team has more minion kills, your team probably has higher experience level because minion kills give experience. 

# Conclusion

This data reinforces what most League of Legend players believe: "Get more kills, try not to die, make sure you farm minions, and help your teammates!"

It is worth noting that players "throwing" the game is always a possibility. This means that a team has a massive lead (either many more kills or many more objectives) could "throw" the game by misplaying and giving the opposite team an advantage. During this advantage, the opposite team could end the game. With this possibility, the losing team could have more kills, more objectives, and more experience but still lose. This could explain why the best performing model only has 73% since it cannot predict a team that "throws" the game.

Thank you for reading. This was my first Kaggle notebook and any feedback is appreciated!